In [1]:
#learn kung fu
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import mason_functions as mf

import os
import evaluate
import warnings
warnings.filterwarnings('ignore')

import sklearn.preprocessing
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.feature_selection import SelectKBest, f_regression, RFE

In [2]:
#define my sql query to access the relational database
sql = '''
SELECT * 
FROM predictions_2017
LEFT JOIN properties_2017 USING(parcelid)
WHERE propertylandusetypeid = 261 or propertylandusetypeid = 279
'''

#define my url
url = mf.get_db_url('zillow')

#read the information from a database into a dataframe
#I also will not continuously query the database; instead, I will write the data to a .csv (in case the kernel gotta go)
if os.path.isfile('zillow_actual.csv'):
    df = pd.read_csv('zillow_actual.csv', index_col = 0)
else:
    df = pd.read_sql(sql, url)
    df.to_csv('zillow_actual.csv')

In [3]:
#at a glance
pd.options.display.max_columns = 69
print(df.info)

<bound method DataFrame.info of        parcelid     id  logerror transactiondate     id.1  \
0      14297519      0  0.025595      2017-01-01  1727539   
1      17052889      1  0.055619      2017-01-01  1387261   
2      14186244      2  0.005383      2017-01-01    11677   
3      12177905      3 -0.103410      2017-01-01  2288172   
4      12095076      6 -0.001011      2017-01-01   781532   
...         ...    ...       ...             ...      ...   
52437  11000655  77609  0.020615      2017-09-20   673515   
52438  17239384  77610  0.013209      2017-09-21  2968375   
52439  12773139  77611  0.037129      2017-09-21  1843709   
52440  12826780  77612  0.007204      2017-09-25  1187175   
52441  13083743  77613 -0.197755      2018-05-25  2455685   

       airconditioningtypeid  architecturalstyletypeid  basementsqft  \
0                        NaN                       NaN           NaN   
1                        NaN                       NaN           NaN   
2                  

In [4]:
#double-take
df.head()

,parcelid,id,logerror,transactiondate,id.1,airconditioningtypeid,architecturalstyletypeid,basementsqft,bathroomcnt,bedroomcnt,buildingclasstypeid,buildingqualitytypeid,calculatedbathnbr,decktypeid,finishedfloor1squarefeet,calculatedfinishedsquarefeet,finishedsquarefeet12,finishedsquarefeet13,finishedsquarefeet15,finishedsquarefeet50,finishedsquarefeet6,fips,fireplacecnt,fullbathcnt,garagecarcnt,garagetotalsqft,hashottuborspa,heatingorsystemtypeid,latitude,longitude,lotsizesquarefeet,poolcnt,poolsizesum,pooltypeid10,pooltypeid2,pooltypeid7,propertycountylandusecode,propertylandusetypeid,propertyzoningdesc,rawcensustractandblock,regionidcity,regionidcounty,regionidneighborhood,regionidzip,roomcnt,storytypeid,threequarterbathnbr,typeconstructiontypeid,unitcnt,yardbuildingsqft17,yardbuildingsqft26,yearbuilt,numberofstories,fireplaceflag,structuretaxvaluedollarcnt,taxvaluedollarcnt,assessmentyear,landtaxvaluedollarcnt,taxamount,taxdelinquencyflag,taxdelinquencyyear,censustractandblock
0,14297519,0,0.025595,2017-01-01,1727539,NaN,NaN,NaN,3.5,4.0,NaN,NaN,3.5,NaN,NaN,3100.0,3100.0,NaN,NaN,NaN,NaN,6059.0,NaN,3.0,2.0,633.0,NaN,NaN,33634931.0,-117869207.0,4506.0,NaN,NaN,NaN,NaN,NaN,122,261.0,NaN,6.059063e+07,53571.0,1286.0,NaN,96978.0,0.0,NaN,1.0,NaN,NaN,NaN,NaN,1998.0,NaN,NaN,485713.0,1023282.0,2016.0,537569.0,11013.72,NaN,NaN,6.059063e+13
1,17052889,1,0.055619,2017-01-01,1387261,NaN,NaN,NaN,1.0,2.0,NaN,NaN,1.0,NaN,1465.0,1465.0,1465.0,NaN,NaN,1465.0,NaN,6111.0,1.0,1.0,1.0,0.0,NaN,NaN,34449266.0,-119281531.0,12647.0,NaN,NaN,NaN,NaN,NaN,1110,261.0,NaN,6.111001e+07,13091.0,2061.0,NaN,97099.0,5.0,NaN,NaN,NaN,NaN,NaN,NaN,1967.0,1.0,NaN,88000.0,464000.0,2016.0,376000.0,5672.48,NaN,NaN,6.111001e+13
2,14186244,2,0.005383,2017-01-01,11677,NaN,NaN,NaN,2.0,3.0,NaN,NaN,2.0,NaN,NaN,1243.0,1243.0,NaN,NaN,NaN,NaN,6059.0,NaN,2.0,2.0,440.0,NaN,NaN,33886168.0,-117823170.0,8432.0,1.0,NaN,NaN,NaN,1.0,122,261.0,NaN,6.059022e+07,21412.0,1286.0,NaN,97078.0,6.0,NaN,NaN,NaN,NaN,NaN,NaN,1962.0,1.0,NaN,85289.0,564778.0,2016.0,479489.0,6488.30,NaN,NaN,6.059022e+13
3,12177905,3,-0.103410,2017-01-01,2288172,NaN,NaN,NaN,3.0,4.0,NaN,8.0,3.0,NaN,NaN,2376.0,2376.0,NaN,NaN,NaN,NaN,6037.0,NaN,3.0,NaN,NaN,NaN,2.0,34245180.0,-118240722.0,13038.0,1.0,NaN,NaN,NaN,1.0,0101,261.0,LCR110000*,6.037300e+07,396551.0,3101.0,NaN,96330.0,0.0,NaN,NaN,NaN,1.0,NaN,NaN,1970.0,NaN,NaN,108918.0,145143.0,2016.0,36225.0,1777.51,NaN,NaN,6.037300e+13
4,12095076,6,-0.001011,2017-01-01,781532,1.0,NaN,NaN,3.0,4.0,NaN,9.0,3.0,NaN,NaN,2962.0,2962.0,NaN,NaN,NaN,NaN,6037.0,NaN,3.0,NaN,NaN,NaN,2.0,34145202.0,-118179824.0,63000.0,1.0,NaN,NaN,NaN,1.0,0101,261.0,PSR2,6.037461e+07,47019.0,3101.0,274684.0,96293.0,0.0,NaN,NaN,NaN,1.0,NaN,NaN,1950.0,NaN,NaN,276684.0,773303.0,2016.0,496619.0,9516.26,NaN,NaN,6.037461e+13
